In [1]:
import icechunk
import zarr

Lets create an in-memory icechunk store

In [2]:
repo = icechunk.Repository.create(
    storage=icechunk.in_memory_storage()
)
repo

Ok! Lets create some data!

In [4]:
session = repo.writable_session("main")
group = zarr.group(store=session.store, overwrite=True)
group

<Group <icechunk.store.IcechunkStore object at 0x1053c55d0>>

In [6]:
air_temp = group.create_array(
    "air_temp", shape=(1000, 1000), chunks=(100, 100), dtype="i4"
)
air_temp

<Array <icechunk.store.IcechunkStore object at 0x1053c55d0>/air_temp shape=(1000, 1000) dtype=int32>

In [8]:
async for key in session.store.list():
    print(key)

zarr.json
air_temp/zarr.json


In [9]:
air_temp[:, :] = 42

In [10]:
air_temp[200, 6]

array(42, dtype=int32)

Now that we have set the values, lets commit

In [11]:
snapshot_id = session.commit("Initial commit")
snapshot_id

'VRW71ARWS4QF5R2S7SQ0'

Lets get another session

In [12]:
from typing import cast


session = repo.writable_session("main")
group = zarr.open_group(session.store)
air_temp = cast(zarr.Array, group["air_temp"])

Okay now we can change the data

In [13]:
air_temp[:, :] = 54

In [14]:
air_temp[200, 6]

array(54, dtype=int32)

And we can commit again

In [15]:
new_snapshot_id = session.commit("Change air temp to 54")
new_snapshot_id

'W8VV15HGFVJ40XTFNCBG'

Cool, now lets checkout the original snapshot and see if the value is 42 again

In [16]:
group = zarr.open_group(session.store, mode='r')
air_temp = cast(zarr.Array, group["air_temp"])
air_temp[200, 6]


array(54, dtype=int32)